# Merging EPO treatment and location data

This workbook attempts to merge the patent data with the location data.

The patent data comes from `EPO_cited&citing_noself.csv` which contains application number and application id's for both cited and citing patents. Cited patents refer to patents located in the UK that have subseuqnetly been cited in the EPO dataset. Citing patents refer to those that cite these patents. 

This patent data is merged with the individual data from the EPO dataset to identify the publication numbers for both the citing and citing patent. These publication numbers differ from the location data in the `first and subsequent` data as they do not have the `EP` string at the beginning. This is subsequently added to align them with the location data.

For preparation for the merge with the location data, the patent data is merged with the `first and subsequent` dataset along the publication number for both the citing and the cited dataset. Both of these merges result in a retention of all individualc cited-citing pairs with the `appln_id`. This is then subseuqnelty merged with the `geo_inv.txt` data along the lines of the `appln_id` however both merges result in a reduction in the overall amount of data from 68,996 to 19,478. While it is recognised that the bounds of the patent information exceeds the time bounds of the location data by ranging from 1977 to 2019, such a big drop off in data is to be expected. 

In [1]:
#importing the necessary libraries to complete the work
import pandas as pd
import numpy as np

In [2]:
#read in both the treatment and EPO_ind data
#EPO_treatment is the data to be used for the study, after creating cited and citing pairs 
#from the EPO dataset in REGPAT 2019. Self citations are removed.
EPO_treatment = pd.read_csv("Patents data/EPO_cited&citing_noself.csv")

#EPO_ind data comes from the REGPAT database 
#this is to be merged with the treatment data to get the publication number for both the cited and 
#citing patents
EPO_ind = pd.read_csv("Patents data/202001_EPO_Inv_reg.txt", sep ="|", header = 0)

In [3]:
#make sure the results are read in correctly
EPO_ind

,app_nbr,appln_id,pub_nbr,person_id,inv_name,address,reg_code,ctry_code,reg_share,inv_share
0,EP20000103094,1,1037159,2,"Lipponen, Markku","Simo Kaarion katu 1 A 2,33720 Tampere",FI197,FI,1.0,0.250000
1,EP20000103094,1,1037159,3,"Laitinen, Timo","Peiponkatu 6,37830 Viiala",FI197,FI,1.0,0.250000
2,EP20000103094,1,1037159,4,"Aho, Ari","Elementinpolku 13 A 6,33720 Tampere",FI197,FI,1.0,0.250000
3,EP20000103094,1,1037159,5,"Knuutila, Jarno","Matti Tapion katu 1 F 17,33720 Tampere",FI197,FI,1.0,0.250000
4,EP20000107845,2,1024191,9,"Griffiths, Andrew David","28 Lilac Court, Cherry Hinton Road,Cambridge C...",UKH12,GB,1.0,0.166667
5,EP20000107845,2,1024191,10,"Hoogenboom, Hendricus Renerus Jacobus Mattheus","1 Hauxton Road, Little Shelford,Cambridge CB2 5JH",UKH12,GB,1.0,0.166667
6,EP20000107845,2,1024191,11,"Marks, James David","107 Ardmore,Kesington, CA 94707",US06001,US,1.0,0.166667
7,EP20000107845,2,1024191,12,"McCafferty, John","32 Wakelin Avenue,Sawston, Cambridgeshire CB2 4DA",UKH12,GB,1.0,0.166667
8,EP20000107845,2,1024191,13,"Winter, Gregory Paul","c/o Trinity College,Cambridge CB2 1TQ",UKH12,GB,1.0,0.166667
9,EP20000107845,2,1024191,14,"Grigg, Geoffrey Walter","352 Burns Bay Road, Lane Cove,Linley Point, NS...",AU1GS,AU,1.0,0.166667


In [4]:
#the same with the treatment data
EPO_treatment

,Unnamed: 0,Citing_app_nbr,Citing_appln_id,Cited_App_nbr,Cited_Appln_id,prio_year,Citing_IPC,IPC_subclass,Cited_firm_id,citing_firm_id,Cited_ind_id,citing_ind_id
0,0,EP19780101388,16428374,EP19780300148,16429355.0,1977,A61K031/43,A61K,[28827],[1386114],"[2218534, 2218535, 2218536, 2218537]","[2216665, 2216666]"
1,1,EP19790301944,16438624,EP19780300148,16429355.0,1978,A61K031/41,A61K,[28827],[2236098],"[2218534, 2218535, 2218536, 2218537]",[2236179]
2,2,EP19790302384,16439056,EP19780300148,16429355.0,1978,A61K031/43,A61K,[28827],[11539],"[2218534, 2218535, 2218536, 2218537]",[2237044]
3,3,EP19790302384,16439056,EP19790300096,16436815.0,1978,A61K031/43,A61K,[28827],[11539],[2218537],[2237044]
4,4,EP19820305125,16503864,EP19780300148,16429355.0,1981,A61K031/395,A61K,[28827],[209914],"[2218534, 2218535, 2218536, 2218537]","[2341206, 2347075, 2347076]"
5,5,EP19880810407,16740869,EP19780300148,16429355.0,1987,A61K031/43,A61K,[28827],[1621],"[2218534, 2218535, 2218536, 2218537]","[2403985, 2521651, 2750251]"
6,6,EP20080163922,201033,EP19790300096,16436815.0,2003,C12Q001/68,C12Q,[28827],[223388],[2218537],"[298714, 298715, 298716, 298717, 298718, 29871..."
7,7,EP19790102118,16432764,EP19780300203,16429410.0,1978,A01N043/40,A01N,[12885],[1621],[2218661],"[1719, 2214349, 2225067, 2225068]"
8,8,EP19790102118,16432764,EP19780300853,16430044.0,1978,A01N043/40,A01N,[12885],[1621],[2219864],"[1719, 2214349, 2225067, 2225068]"
9,9,EP19790103431,16434023,EP19780300203,16429410.0,1978,C07D 213/26,C07D,[12885],[2227427],[2218661],"[2227428, 2227429, 2227430, 2227431, 2227432, ..."


In [5]:
#for the EPO_ind data we want to group by the app_nbr
#we want the first result for each of the columns as the publication number should be the same
EPO_ind = EPO_ind.groupby("app_nbr").first().reset_index()

In [6]:
#drop any unecessary rows so as to save on computing power and increase teh speed of the code
EPO_ind.drop(["appln_id","inv_name", "address", "reg_code", "ctry_code", "reg_share", 
              "inv_share", "person_id"], axis =1, inplace = True)

In [7]:
#check the results
EPO_ind

,app_nbr,pub_nbr
0,EP19780100001,113
1,EP19780100002,1379
2,EP19780100003,5695
3,EP19780100004,323
4,EP19780100005,114
5,EP19780100006,5696
6,EP19780100007,2
7,EP19780100008,6400
8,EP19780100009,10
9,EP19780100011,859


In [8]:
#merge the treatment data with the EPO_ind data
#this is to get the publication number for the citing patents 
EPO_treatment = EPO_treatment.merge(EPO_ind, left_on = "Citing_app_nbr", right_on= "app_nbr")

In [9]:
#subsequently uncessary columns can be dropped
EPO_treatment.drop(["Citing_IPC", "IPC_subclass", "Cited_firm_id",
                    "citing_firm_id", "Cited_ind_id", "citing_ind_id", 
                    "app_nbr"], axis = 1, inplace = True)
#the publication number can be renamed to attach it to the citing patent
EPO_treatment.rename(columns = {"pub_nbr":"Citing_pub_nbr"}, inplace = True)

In [10]:
#the unnamed: 0 column can be used to check for loss of rows and/or results
#this is because it acts as an index for every result so that a reduction in this value
#means that there is a loss of cited-citing pairs
len(list(EPO_treatment["Unnamed: 0"].unique()))

68996

In [11]:
#this merge and process is repeated for the cited applications 
EPO_treatment = EPO_treatment.merge(EPO_ind, left_on = "Cited_App_nbr", right_on = "app_nbr")

In [12]:
#the columms are renamed to attach it to the cited patent
EPO_treatment.rename(columns = {"pub_nbr":"Cited_pub_nbr"}, inplace = True)
#and the unecessary colummms are dropped
EPO_treatment.drop(["app_nbr"], axis =1, inplace = True)

In [13]:
#Exploring the first and subsequent dataset, the publication number is suggsted to be a combination
#of the patent office and number
#For this purpose the publictaion numbers here have to be changed to strings so that EP can be 
#added to the beginning of the string
EPO_treatment = EPO_treatment.astype({"Citing_pub_nbr": str, "Cited_pub_nbr": str})

In [14]:
#Here the EP is added to the beginning of both the publication numbers for the cited and citing 
#patents
EPO_treatment["Cited_pub_nbr"] = "EP" + EPO_treatment["Cited_pub_nbr"].astype(str)
EPO_treatment["Citing_pub_nbr"] = "EP" + EPO_treatment["Citing_pub_nbr"].astype(str)

In [15]:
#any unecessary columns are dropped
EPO_treatment.drop(["prio_year"], axis = 1, inplace = True)

In [16]:
#this outputs the resulting dataframe to a csv so that it can be recovered later
EPO_treatment.to_csv("EPO_treatment_data.csv", index = False)

## after pre-processing

To reduce the data hosting/transfer requirements only the merged dataset is uploaded with all the preprocessing already done from the above code. This can then be read in as EPO_treatment in the following code block.

In [17]:
#this reads in the subsequent csv
EPO_treatment = pd.read_csv("EPO_treatment_data.csv")
#the data can then be explored
EPO_treatment

,Unnamed: 0,Citing_app_nbr,Citing_appln_id,Cited_App_nbr,Cited_Appln_id,Citing_pub_nbr,Cited_pub_nbr
0,0,EP19780101388,16428374,EP19780300148,16429355.0,EP2210,EP636
1,1,EP19790301944,16438624,EP19780300148,16429355.0,EP10358,EP636
2,2,EP19790302384,16439056,EP19780300148,16429355.0,EP13067,EP636
3,4,EP19820305125,16503864,EP19780300148,16429355.0,EP76621,EP636
4,5,EP19880810407,16740869,EP19780300148,16429355.0,EP297042,EP636
5,3,EP19790302384,16439056,EP19790300096,16436815.0,EP13067,EP3415
6,6,EP20080163922,201033,EP19790300096,16436815.0,EP2050827,EP3415
7,7,EP19790102118,16432764,EP19780300203,16429410.0,EP6608,EP1473
8,9,EP19790103431,16434023,EP19780300203,16429410.0,EP9212,EP1473
9,10,EP19800304466,16456262,EP19780300203,16429410.0,EP31218,EP1473


In [18]:
#the first and subsequent data has already been preprocessed to get only results from the EPO
#this is after concatenating the f&S 1 & 2 dataframes into a single dataframe and indexing based on
#publn_auth == "EP"
fS_EP = pd.read_csv("Patents data/first_and_subsequent_EP.csv")

C:\Users\cex\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
#the unecessary columns are dropped
fS_EP.drop(["Unnamed: 0"], axis = 1, inplace = True)

In [20]:
#the data is explored
fS_EP

,appln_id,is_first,publn_auth,publn_nr,publn_nr_original,publn_kind
0,13005743,0,EP,1304202,NaN,B1
1,37537365,0,EP,1316461,NaN,B1
2,46068065,0,EP,1159961,NaN,A3
3,17,1,EP,1345702,NaN,A1
4,17,1,EP,1345702,NaN,B1
5,37,1,EP,1361052,NaN,A2
6,37,1,EP,1361052,NaN,A3
7,37,1,EP,1361052,NaN,B1
8,58,1,EP,1643444,NaN,A1
9,58,1,EP,1643444,NaN,B1


In [21]:
#to get the publication number to match that of the treatment data the publication auth and number
#are merged to form a string
fS_EP["EPO_pub_nbr"] = fS_EP["publn_auth"] + fS_EP["publn_nr"].astype(str)

In [22]:
#this groupby was originally performed to limit the amount of results that were received 
#but the final number of results increased after this was removed
#fS_EP = fS_EP.groupby("appln_id").first().reset_index()

In [23]:
#the treatment dataset is merged with the forst and subsequent dataset based on the publication number
#this is first performed on the cited patents
EPO_treatment = EPO_treatment.merge(fS_EP, left_on = "Cited_pub_nbr", right_on = "EPO_pub_nbr")

In [24]:
#the resulting number is checked 
EPO_treatment.count()
#this shows that each citing-cited pair has more than one merge partner in the fS_EP dataframe

#to check the result in terms of how many citing-cited pairs remain
#this shows that most are retained after this merge
len(list(EPO_treatment["Unnamed: 0"].unique()))

68994

In [25]:
#any unecessary columns are subsequently dropped 
EPO_treatment.drop(["publn_auth", "publn_nr", "publn_nr_original", "publn_kind"], 
                  axis =1, inplace = True)

In [26]:
#read in the geo_inv dataset
geoc_inv = pd.read_csv("Patents data/geoc_inv_person.txt", sep = ",", header = 0)

C:\Users\cex\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [27]:
#explore the results
geoc_inv

,appln_id,person_id,patent_office,filing_date,lat,lng,ctry_code,name_0,name_1,name_2,name_3,name_4,name_5,city,coord_source,source,type
0,1,4,EP,2000-02-15,61.4431,23.851900,FI,Finland,Western Finland,Pirkanmaa,Tampere,Tampere,NaN,Tampere,geolocalization,1,continual
1,1,2,EP,2000-02-15,61.4446,23.871900,FI,Finland,Western Finland,Pirkanmaa,Tampere,Tampere,NaN,Tampere,geolocalization,1,continual
2,1,3,EP,2000-02-15,61.2128,23.739000,FI,Finland,Western Finland,Pirkanmaa,Southern Pirkanmaa,Viiala,NaN,Viiala,geolocalization,1,continual
3,1,5,EP,2000-02-15,61.4416,23.865900,FI,Finland,Western Finland,Pirkanmaa,Tampere,Tampere,NaN,Tampere,geolocalization,1,continual
4,2,13,EP,1992-12-02,52.2070,0.113082,GB,United Kingdom,England,Cambridgeshire,South Cambridgeshire,NaN,NaN,South Cambridgeshire,geolocalization,1,continual
5,2,11,EP,1992-12-02,37.9048,-122.281000,US,United States,California,Contra Costa,NaN,NaN,NaN,Contra Costa,geolocalization,1,continual
6,2,14,EP,1992-12-02,-33.8246,151.151000,AU,Australia,New South Wales,Lane Cove,NaN,NaN,NaN,Lane Cove,geolocalization,1,continual
7,2,10,EP,1992-12-02,52.1712,0.112405,GB,United Kingdom,England,Cambridgeshire,South Cambridgeshire,NaN,NaN,South Cambridgeshire,geolocalization,1,continual
8,2,9,EP,1992-12-02,52.2037,0.124092,GB,United Kingdom,England,Cambridgeshire,South Cambridgeshire,NaN,NaN,South Cambridgeshire,geonames,1,continual
9,37,152,EP,2000-12-01,49.8571,9.840940,DE,Germany,Bayern,Würzburg,Margetshöchheim,Erlabrunn,NaN,Erlabrunn,geolocalization,1,priority


In [28]:
#merge with the EPO_treatment dataset, which has already been merged with the fS_EPO dataset
#this is merged based on appln_id
EPO_treatment_merged = EPO_treatment.merge(geoc_inv, on = "appln_id")
#EPO_treatment_merged = EPO_treatment.merge(geoc_inv, left_on = "Cited_Appln_id", right_on = "appln_id")

In [29]:
#drop any unecessary columns
EPO_treatment_merged.drop(["name_3", "name_4", "coord_source", "source", "type"], axis =1, inplace = True)

In [30]:
#check the length of the list of unnamed: 0  column in the resulting dataframe
#this suggests merging based on appln_id results in a los of #40,000 results
#suggesting that 40,000 of the cited patents are not in the geo_inv dataframe
len(list(EPO_treatment_merged["Unnamed: 0"].unique()))

28315

In [31]:
#the results can be explored here
EPO_treatment_merged

,Unnamed: 0,Citing_app_nbr,Citing_appln_id,Cited_App_nbr,Cited_Appln_id,Citing_pub_nbr,Cited_pub_nbr,appln_id,is_first,EPO_pub_nbr,...,patent_office,filing_date,lat,lng,ctry_code,name_0,name_1,name_2,name_5,city
0,69,EP19840304274,16564703,EP19830304043,16531258.0,EP140467,EP101181,16531258,1,EP101181,...,EP,1983-07-12,51.6205,-0.513980,GB,United Kingdom,England,Hertfordshire,NaN,Three Rivers
1,69,EP19840304274,16564703,EP19830304043,16531258.0,EP140467,EP101181,16531258,1,EP101181,...,EP,1983-07-12,51.6205,-0.513980,GB,United Kingdom,England,Hertfordshire,NaN,Three Rivers
2,5291,EP19950111004,17097631,EP19830304043,16531258.0,EP693542,EP101181,16531258,1,EP101181,...,EP,1983-07-12,51.6205,-0.513980,GB,United Kingdom,England,Hertfordshire,NaN,Three Rivers
3,5291,EP19950111004,17097631,EP19830304043,16531258.0,EP693542,EP101181,16531258,1,EP101181,...,EP,1983-07-12,51.6205,-0.513980,GB,United Kingdom,England,Hertfordshire,NaN,Three Rivers
4,90,EP19890307758,16782747,EP19810301983,16475271.0,EP353973,EP40038,16475271,1,EP40038,...,EP,1981-05-06,55.0058,-1.625700,GB,United Kingdom,England,Newcastle upon Tyne,NaN,Newcastle upon Tyne
5,90,EP19890307758,16782747,EP19810301983,16475271.0,EP353973,EP40038,16475271,1,EP40038,...,EP,1981-05-06,55.0058,-1.625700,GB,United Kingdom,England,Newcastle upon Tyne,NaN,Newcastle upon Tyne
6,90,EP19890307758,16782747,EP19810301983,16475271.0,EP353973,EP40038,16475271,1,EP40038,...,EP,1981-05-06,55.0058,-1.625700,GB,United Kingdom,England,Newcastle upon Tyne,NaN,Newcastle upon Tyne
7,94,EP19840308166,16568465,EP19810301983,16475271.0,EP143629,EP40038,16475271,1,EP40038,...,EP,1981-05-06,55.0058,-1.625700,GB,United Kingdom,England,Newcastle upon Tyne,NaN,Newcastle upon Tyne
8,94,EP19840308166,16568465,EP19810301983,16475271.0,EP143629,EP40038,16475271,1,EP40038,...,EP,1981-05-06,55.0058,-1.625700,GB,United Kingdom,England,Newcastle upon Tyne,NaN,Newcastle upon Tyne
9,94,EP19840308166,16568465,EP19810301983,16475271.0,EP143629,EP40038,16475271,1,EP40038,...,EP,1981-05-06,55.0058,-1.625700,GB,United Kingdom,England,Newcastle upon Tyne,NaN,Newcastle upon Tyne


In [32]:
#this is then repeated for the citing information seperately
EPO_treatment2 = EPO_treatment.merge(fS_EP, left_on = "Citing_pub_nbr", right_on = "EPO_pub_nbr")
EPO_treatment_merged2 = EPO_treatment2.merge(geoc_inv, left_on  = "appln_id_y", right_on = "appln_id")
#this suggests that when looking at the citing publications, there is only a loss of around
#20,000 results
len(list(EPO_treatment_merged2["Unnamed: 0"].unique()))

46399

In [33]:
#the loss of total results can also be exploed after following the cited and then the citing
EPO_treatment_merged = EPO_treatment_merged.merge(fS_EP, left_on = "Citing_pub_nbr", right_on = "EPO_pub_nbr")

In [34]:
EPO_treatment_merged = EPO_treatment_merged.merge(geoc_inv, left_on = "appln_id_y", right_on ="appln_id")
#EPO_treatment_merged = EPO_treatment_merged.merge(geoc_inv, left_on = "Citing_appln_id", right_on ="appln_id")

In [35]:
#if both cited and citing and geocoded then there is loss in the order of #48,000 cited-citing pairs
len(list(EPO_treatment_merged["Unnamed: 0"].unique()))

19487